In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
from keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, TerminateOnNaN, EarlyStopping
from tensorflow.keras import optimizers
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.TPUStrategy(resolver)

path = '/content/drive/MyDrive/Colab Notebooks/'
def load_data(name, s=0):
    
    X = np.load(path+'X_%s.npy'%name)
    Y = np.load(path+'Y_%s.npy'%name)
    if s == 0: return X, Y
        
    #나누기
    X_b = X[-s:]
    Y_b = Y[-s:]

    X_f = X[:-s]
    Y_f = Y[:-s]

    return X_f, Y_f, X_b, Y_b


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.17.126.186:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.17.126.186:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
!pip install tfa-nightly
import tensorflow_addons as tfa
!pip install focal-loss
from focal_loss import BinaryFocalLoss

In [7]:
list = np.array(('3bull_1h_3','train_1', 'etf_1h_8' , 'train_2'))
#X_train, Y_train = load_data('Bitcoin_1h_5')

#list = np.array(('stock_1h_7_1', 'stock_1h_7_2', 'etf_1h_7', 'stock_1h_7_3'))

#container = {}
#for name in list:
#  container[name] = load_data(name)



In [5]:
X_val, Y_val = load_data('test')
size=np.shape(Y_val)[0]

In [9]:
#sel = [0,1,2,4,5,6]

lr=2e-3
with strategy.scope():
#while True:
  model = load_model(path+'GRU2_1h.h5')
  model.summary()
  model.trainable = True
#  for i in range(0, 5, 2): model.layers[i].trainable = False
  

  
  while True:    
    for name in list:
      lr*=0.95
      #I = np.arange(size)  #표본 데이터가 너무 많아서 추출하고 싶을 때 쓰는 인덱스
      #np.random.shuffle(I)
      
      K.clear_session()
      X_train, Y_train = load_data(name)
#      X_train, Y_train = container[name]
#      X_train, Y_train = load_data('train_Bitcoin_1h_3')

      print('\n\n#'+name)

#      fit_values={"stock":}
#      p = fit_values[name.split('_')[1]]

#      optimizer = tfa.optimizers.RectifiedAdam(learning_rate=lr)   
      optimizer = tfa.optimizers.AdamW(weight_decay=1e-3, learning_rate=lr)
      p=1
      
      model.compile(optimizer=optimizer,
#                    loss="binary_crossentropy",
                    loss=BinaryFocalLoss(gamma=1),
                    metrics=['acc'])


      model.fit(X_train,
                Y_train,
                epochs=128,
                batch_size=8192,
                validation_data=(X_val, Y_val),
                callbacks=[ModelCheckpoint(path+'GRU2_1h_{val_loss:.4f}_{val_acc:.4f}.h5',
                                           monitor="val_loss",
                                           verbose=0,
                                           save_best_only=True,
                                           save_weights_only=False,
                                           save_freq = 'epoch'
                                           ),
                           EarlyStopping(monitor='val_loss', min_delta=0, patience=p, verbose=0, mode='auto', baseline=None, restore_best_weights=True),
                           TerminateOnNaN()])
      
      
                            
      #del X_train, Y_train

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 210, 32)           4736      
_________________________________________________________________
dropout (Dropout)            (None, 210, 32)           0         
_________________________________________________________________
gru (GRU)                    (None, 210, 32)           6336      
_________________________________________________________________
dropout_1 (Dropout)          (None, 210, 32)           0         
_________________________________________________________________
gru_1 (GRU)                  (None, 8)                 1008      
_________________________________________________________________
dropout_2 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense (Dense)                (None, 1)                 9

25/25 [==============================] - 31s 859ms/step - loss: 0.4697 - acc: 0.5123 - val_loss: 0.4360 - val_acc: 0.4838
Epoch 2/128
25/25 [==============================] - 15s 593ms/step - loss: 0.4294 - acc: 0.5203 - val_loss: 0.4223 - val_acc: 0.4677
Epoch 3/128
25/25 [==============================] - 14s 581ms/step - loss: 0.4114 - acc: 0.5287 - val_loss: 0.4107 - val_acc: 0.4553
Epoch 4/128
25/25 [==============================] - 14s 578ms/step - loss: 0.3990 - acc: 0.5315 - val_loss: 0.4014 - val_acc: 0.4453
Epoch 5/128
25/25 [==============================] - 14s 578ms/step - loss: 0.3893 - acc: 0.5380 - val_loss: 0.3939 - val_acc: 0.4423
Epoch 6/128
25/25 [==============================] - 14s 578ms/step - loss: 0.3816 - acc: 0.5378 - val_loss: 0.3879 - val_acc: 0.4112
Epoch 7/128
25/25 [==============================] - 14s 578ms/step - loss: 0.3750 - acc: 0.5414 - val_loss: 0.3830 - val_acc: 0.4370
Epoch 8/128
25/25 [==============================] - 14s 581ms/step - loss

IndexError: ignored

비트코인은 2층이 최고다 그리고 64의 길이가 최고다. 반면 주식은 이런 길이면 예측 정확도가 0%로 수렴한다.
비트코인의 적절한 수치는 이렇다:
base = 64  # 몇 일의 데이터를 쓸 것인가
period = 21   # 몇 step내를 예측할 것인가
step = 5  # 몇 step씩 이동할 것인가
percentage = 3% #몇 퍼센트를 기준으로 할 것인가

RNN에서 긴 데이터는 곧 자기 규제를 의미한다. 또는 선형화를 의미한다.
주식은 너무나도 넓은 스펙트럼이기 때문에 비트코인보다 긴 길이가 필요한 것 같다.

너무 큰 모델은 AdamW로 하다가 Local Optima에 빠지면 노답이 된다. RAdam으로 학습시킨 후 AdamW를 쓰자. 반면 너무 큰 모델은 Loss폭발 문제도 있다. 초반에 Loss를 정상적 수준으로 떨어뜨리기까지는 AdamW를 쓰자

바이너리 크로스 앤트로피의 계산식을 고려하면 0.5의 확률로 맞출 때 -log(0.5) = log(2) = 0.6931이다. 즉, val_loss가 이 값이면 output은 평균적으로 정답과 0.5만큼 떨어져 있는 것이다.  따라서 이 값보다 작아야 신뢰할만하다고 할 수 있다.

focal loss에 대해서는 결과에 2를 곱한 게 그냥 binary의 loss와 같다. (감마=1에서)

-log(0.7) = -0.3566

 -log(0.65) = 0.4307   
 -log(0.6) = 0.5108   
 -log(0.55) = 0.5978

RNN들에는 같은 가중치가 반복해서 사용된다. 내 생각엔 여기에는 linear적인 문제가 있다. 데이터 길이를 짧게하는 것이 오히려 더 나을 수도 있다는 생각에 70개의 연속된 데이터로 바꿨다. 50개로 바꿔서 다시해볼까?

데이터가 조금 쌓여서 최근 데이터로 테스트 데이터를 구성했다.
396개인데, 100년 동안 표본오차 5%에 대해 신뢰도 95%임을 의미한다.
즉 여기서 나온 정확도에 5%정도 차이가 난다고 보면 된다. 계산은 https://www.nownsurvey.com/calculator/에서 했다.

down예측도 정확도만 지나치게 높아서 결과의 신뢰성이 떨어진다. focal_loss를 도입했다.

로그 수익률은 50% 정확도도 안 나오는데, 이는 차분이 고려될 수 없는 구조기 때문인 것 같다.(신호와 시스템)